In [1]:
import os
# numerical computing package
import numpy as np
# 
from scipy import stats
# command style functions that make matplotlib work like MATLAB
import matplotlib.pyplot as plt
# matplotlib
import matplotlib
# find nearest value
def find_nearest_index(array, value):
    return (np.abs(array - value)).argmin()
#
import sys
#
import pickle
#
import xarray as xr
#
from glob import glob as gb
#
import time
#
import cartopy.crs as ccrs
#
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import cosima_cookbook as cc
import pandas as pd
import scipy.interpolate
import scipy.ndimage
from tqdm import tqdm_notebook
import IPython.display
import cmocean as cm
from scipy.io import loadmat
from dask.distributed import Client
import cartopy.crs as ccrs
import cartopy.feature as cft
import matplotlib.ticker as mticker
from glob import glob as gb
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

session = cc.database.create_session()

Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
!pwd

figures_path = '/home/561/erd561/figures/JRA_IAF_v13/'
script_name = nb_name[:-6]
print('Figures will be saved in: ' + figures_path + script_name + '\n')

data_path = '/g/data/e14/erd561/'

years = arange(1993,2017,1)

/home/561/erd561/wombat_jra-iaf_mom025


NameError: name 'nb_name' is not defined

In [ ]:
%%time

from dask.distributed import Client
c = Client()
print(c)

In [ ]:
input_path = '/g/data/e14/erd561/NOAA_OI_SST_V2/'

years_SST = np.arange(1982,2018)

lonW = 100
lonE = 170

latS = -60
latN = -20

sst = xr.open_dataset(input_path + 'sst.mnmean.nc').\
                sst.\
                sel(lon=slice(lonW,lonE)).\
                sel(lat=slice(latS,latN,-1)).\
                groupby('time.year').\
                mean('time', skipna=True).\
                sel(year=slice(1982,2017))

sst = sst.assign_coords(lon=sst.lon)

mask = xr.open_dataset(input_path + 'lsmask.nc').\
                mask.\
                sel(lon=slice(lonW,lonE)).\
                sel(lat=slice(latS,latN,-1))

mask = mask.assign_coords(lon=mask.lon)

sst = sst.where(mask==1, np.nan)

sst = sst.squeeze()
print(sst)

sea_surface_temperature = sst

In [ ]:
# sea_surface_temperature_path = data_path + 'IMOS/IMOS_aggregation_SSTAARS_20181023T031550Z.nc'
# for Y in years:
#     y = int(Y)
#     if y == 1993:
#         sea_surface_temperature = xr.open_dataset(
#         sea_surface_temperature_path).sea_surface_temperature.\
#         sel(time=slice(str(y) + '-01-01', str(y+1) + '-01-01')).\
#         sel(lon=arange(100,170,0.25), method='nearest').\
#         sel(lat=arange(-20,-60,-0.25), method='nearest').\
#         mean(dim='time').assign_coords(year=y)
#     else:
#         sea_surface_temperature = xr.concat([
#         sea_surface_temperature,
#         xr.open_dataset(sea_surface_temperature_path).sea_surface_temperature.\
#         sel(time=slice(str(y) + '-01-01', str(y+1) + '-01-01')).\
#         sel(lon=arange(100,170,0.25), method='nearest').\
#         sel(lat=arange(-20,-60,-0.25), method='nearest').\
#         mean(dim='time').assign_coords(year=y)
#         ], 'year')
#     print(y)
# print(sea_surface_temperature)

In [ ]:
AVISO_list = sorted(gb(data_path + 'AVISO/dt_global_allsat_msla_h_y*.nc'))
AVISO_list

GSLA = xr.open_mfdataset(AVISO_list).sla.\
        sel(lon=slice(100,170)).\
        sel(lat=slice(-60,-20)) *100

print(GSLA)

GSLA = GSLA.sel(time=slice('1993-01-01', '2017-12-31')).groupby('time.year').mean('time')
GSLA.load()
print(GSLA)

In [ ]:
years_mod = np.arange(2218, 2258)
years_id = np.arange(2218, 2258, 2)
n_years = np.arange(1978, 2018)

print(years_mod)
print(len(years_mod))

output_n = ["%03d" % i for i in range(130,150)]
print(output_n)
print(len(output_n))

# use common start and end dates for all runs
# tstart = exptdata.clim_tstart
tstart = '1978-01-01 00:00:00'
tend = exptdata.clim_tend

firstyear = pd.to_datetime(tstart).year  # assumes tstart is 1 January!
lastyear = pd.to_datetime(tend).year-1  # assumes tend is 1 January!
yearrange = str(firstyear)+'-'+str(lastyear)
print(yearrange)
print('tstart = ', tstart)
print('tend = ', tend)


latS = -60
latN = -20
lonW = -260
lonE = -190
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
time_units = exptdata.exptdict[ekey]['time_units']
n_files = exptdata.exptdict[ekey]['n_files']
offset = exptdata.exptdict[ekey]['offset']
print(expt)
print(time_units)
print(n_files)
print(offset)
temp_month = cc.querying.getvar(expt=expt,
                                variable='temp',
                                session=session,
                                ncfile='ocean.nc',
                                n=n_files,
                                offset=offset).\
                            sel(yt_ocean=slice(latS,latN)).\
                            sel(xt_ocean=slice(lonW,lonE)).\
                            sel(st_ocean=0,method='nearest')

temp_month.load()

temp = temp_month.groupby('time.year').mean('time')
temp = temp.sel(year=slice(1978,2017))

xt_ocean = temp.xt_ocean
xt_ocean_corrected = xt_ocean + 360
temp = temp.assign_coords(xt_ocean=xt_ocean_corrected)
print(temp)

In [ ]:
eta_t_month = cc.querying.getvar(expt=expt,
                                variable='eta_t',
                                session=session,
                                ncfile='ocean_month.nc',
                                n=n_files,
                                offset=offset).\
                            sel(yt_ocean=slice(latS,latN)).\
                            sel(xt_ocean=slice(lonW,lonE))

eta_t_month.load()

eta_t = eta_t_month.groupby('time.year').mean('time')*100
eta_t = eta_t.sel(year=slice(1978,2017))

xt_ocean = eta_t.xt_ocean
xt_ocean_corrected = xt_ocean + 360
eta_t = eta_t.assign_coords(xt_ocean=xt_ocean_corrected)
print(eta_t)

In [ ]:
%%time
lon = sea_surface_temperature.lon
lat = sea_surface_temperature.lat
time = years_SST
sea_surface_temperature_anom_82 = \
sea_surface_temperature - sea_surface_temperature.mean(dim='year')
sea_surface_temperature_slopes_82 = xr.DataArray(
    np.zeros([lat.shape[0], lon.shape[0]]), dims=(
    'lat', 'lon'), 
    coords=[lat, lon], name='sea_surface_temperature')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):        
        sea_surface_temperature_slopes_82[iid, jid], _, _, _, _ = \
        stats.linregress(time, sea_surface_temperature_anom_82.sel(lon=j, lat=i))
    print('lat ' + str(np.array(i)))
print(sea_surface_temperature_slopes_82)

time = arange(1993,2017)
sea_surface_temperature_anom_93 = \
sea_surface_temperature.sel(year=slice(1993,2017)) - \
sea_surface_temperature.sel(year=slice(1993,2017)).mean(dim='year')
sea_surface_temperature_slopes_93 = xr.DataArray(
    np.zeros([lat.shape[0], lon.shape[0]]), dims=(
    'lat', 'lon'), 
    coords=[lat, lon], name='sea_surface_temperature')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):        
        sea_surface_temperature_slopes_93[iid, jid], _, _, _, _ = \
        stats.linregress(time, sea_surface_temperature_anom_93.sel(lon=j, lat=i))
    print('lat ' + str(np.array(i)))
print(sea_surface_temperature_slopes_93)



temp_anom_93 = temp.sel(year=slice(1993,2017)) - temp.sel(year=slice(1993,2017)).mean(dim='year')
temp_anom_82 = temp.sel(year=slice(1982,2017)) - temp.sel(year=slice(1982,2017)).mean(dim='year')
eta_t_anom = eta_t.sel(year=slice(1993,2017)) - eta_t.sel(year=slice(1993,2017)).mean(dim='year')

yt_ocean = temp.yt_ocean
xt_ocean = temp.xt_ocean
temp_slopes_93 = xr.DataArray(np.zeros([yt_ocean.shape[0], xt_ocean.shape[0]]), dims=(
    'latitude', 'longitude'), coords=[yt_ocean, xt_ocean], name='temp')
for iid, i in enumerate(yt_ocean):
    for jid, j in enumerate(xt_ocean):
        temp_slopes_93[iid, jid], _, _, _, _ = \
        stats.linregress(arange(1993,2017), temp_anom_93.sel(xt_ocean=j, yt_ocean=i))
    print('lat ' + str(np.array(i)))
print(temp_slopes_93)    

yt_ocean = temp.yt_ocean
xt_ocean = temp.xt_ocean
temp_slopes_82 = xr.DataArray(np.zeros([yt_ocean.shape[0], xt_ocean.shape[0]]), dims=(
    'latitude', 'longitude'), coords=[yt_ocean, xt_ocean], name='temp')
for iid, i in enumerate(yt_ocean):
    for jid, j in enumerate(xt_ocean):
        temp_slopes_82[iid, jid], _, _, _, _ = \
        stats.linregress(years_SST, temp_anom_82.sel(xt_ocean=j, yt_ocean=i))
    print('lat ' + str(np.array(i)))
print(temp_slopes_82)   

eta_t_slopes = xr.DataArray(np.zeros([yt_ocean.shape[0], xt_ocean.shape[0]]), dims=(
    'latitude', 'longitude'), coords=[yt_ocean, xt_ocean], name='eta_t')
for iid, i in enumerate(yt_ocean):
    for jid, j in enumerate(xt_ocean):        
        eta_t_slopes[iid, jid], _, _, _, _ = \
        stats.linregress(arange(1993,2017), eta_t_anom.sel(xt_ocean=j, yt_ocean=i))
    print('lat ' + str(np.array(i)))
print(eta_t_slopes)



lon = GSLA.lon
lat = GSLA.lat
time = years
GSLA_anom = GSLA - GSLA.mean(dim='year')
GSLA_slopes = xr.DataArray(
    np.zeros([lat.shape[0], lon.shape[0]]), dims=(
    'lat', 'lon'), 
    coords=[lat, lon], name='GSLA')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):        
        GSLA_slopes[iid, jid], _, _, _, _ = \
        stats.linregress(time, GSLA_anom.sel(lon=j, lat=i))
    print('lat ' + str(np.array(i)))
print(GSLA_slopes)

In [ ]:
lon_SA = [112, 145]
lat_SA = [-41, -31]
sea_surface_temperature_anom_SA_82 = sea_surface_temperature_anom_82.\
sel(lon=slice(lon_SA[0],lon_SA[1])).\
sel(lat=slice(lat_SA[0],lat_SA[1])).\
mean(dim='lon').\
mean(dim='lat')
sea_surface_temperature_anom_SA_slope_82, sea_surface_temperature_anom_SA_intrsct_82, _, _, _ = \
        stats.linregress(years_SST, sea_surface_temperature_anom_SA_82)

sea_surface_temperature_anom_SA_93 = sea_surface_temperature_anom_93.\
sel(lon=slice(lon_SA[0],lon_SA[1])).\
sel(lat=slice(lat_SA[0],lat_SA[1])).\
mean(dim='lon').\
mean(dim='lat')
sea_surface_temperature_anom_SA_slope_93, sea_surface_temperature_anom_SA_intrsct_93, _, _, _ = \
        stats.linregress(arange(1993,2017), sea_surface_temperature_anom_SA_93)


lon_TS = [148, 157]
lat_TS = [-45, -32]
sea_surface_temperature_anom_TS_82 = sea_surface_temperature_anom_82.\
sel(lon=slice(lon_TS[0],lon_TS[1])).\
sel(lat=slice(lat_TS[0],lat_TS[1])).\
mean(dim='lon').\
mean(dim='lat')
sea_surface_temperature_anom_TS_slope_82, sea_surface_temperature_anom_TS_intrsct_82, _, _, _ = \
        stats.linregress(years_SST, sea_surface_temperature_anom_TS_82)
sea_surface_temperature_anom_TS_93 = sea_surface_temperature_anom_93.\
sel(lon=slice(lon_TS[0],lon_TS[1])).\
sel(lat=slice(lat_TS[0],lat_TS[1])).\
mean(dim='lon').\
mean(dim='lat')
sea_surface_temperature_anom_TS_slope_93, sea_surface_temperature_anom_TS_intrsct_93, _, _, _ = \
        stats.linregress(arange(1993,2017), sea_surface_temperature_anom_TS_93)
GSLA_anom_TS = GSLA_anom.\
sel(lon=slice(lon_TS[0],lon_TS[1])).\
sel(lat=slice(lat_TS[0],lat_TS[1])).\
mean(dim='lon').\
mean(dim='lat')
GSLA_anom_TS_slope, GSLA_anom_TS_intrsct, _, _, _ = \
        stats.linregress(years, GSLA_anom_TS)
    
print('OK')

In [ ]:
xt_ocean_SA = [112, 145]
yt_ocean_SA = [-41, -31]
temp_anom_SA_82 = temp_anom_82.\
sel(xt_ocean=slice(xt_ocean_SA[0],xt_ocean_SA[1])).\
sel(yt_ocean=slice(yt_ocean_SA[0],yt_ocean_SA[1])).\
mean(dim='xt_ocean').\
mean(dim='yt_ocean')
temp_anom_SA_slope_82, temp_anom_SA_intrsct_82, _, _, _ = \
        stats.linregress(years_SST, temp_anom_SA_82)

temp_anom_SA_93 = temp_anom_93.\
sel(xt_ocean=slice(xt_ocean_SA[0],xt_ocean_SA[1])).\
sel(yt_ocean=slice(yt_ocean_SA[0],yt_ocean_SA[1])).\
mean(dim='xt_ocean').\
mean(dim='yt_ocean')
temp_anom_SA_slope_93, temp_anom_SA_intrsct_93, _, _, _ = \
        stats.linregress(arange(1993,2017), temp_anom_SA_93)


xt_ocean_TS = [148, 157]
yt_ocean_TS = [-45, -32]
temp_anom_TS_82 = temp_anom_82.\
sel(xt_ocean=slice(xt_ocean_TS[0],xt_ocean_TS[1])).\
sel(yt_ocean=slice(yt_ocean_TS[0],yt_ocean_TS[1])).\
mean(dim='xt_ocean').\
mean(dim='yt_ocean')
temp_anom_TS_slope_82, temp_anom_TS_intrsct_82, _, _, _ = \
        stats.linregress(years_SST, temp_anom_TS_82)
temp_anom_TS_93 = temp_anom_93.\
sel(xt_ocean=slice(xt_ocean_TS[0],xt_ocean_TS[1])).\
sel(yt_ocean=slice(yt_ocean_TS[0],yt_ocean_TS[1])).\
mean(dim='xt_ocean').\
mean(dim='yt_ocean')
temp_anom_TS_slope_93, temp_anom_TS_intrsct_93, _, _, _ = \
        stats.linregress(arange(1993,2017), temp_anom_TS_93)
eta_t_anom_TS = eta_t_anom.\
sel(xt_ocean=slice(xt_ocean_TS[0],xt_ocean_TS[1])).\
sel(yt_ocean=slice(yt_ocean_TS[0],yt_ocean_TS[1])).\
mean(dim='xt_ocean').\
mean(dim='yt_ocean')
eta_t_anom_TS_slope, eta_t_anom_TS_intrsct, _, _, _ = \
        stats.linregress(years, eta_t_anom_TS)
    
print('OK')

In [ ]:
years

In [ ]:
def bbox(pos):
    return matplotlib.transforms.Bbox(np.array([[pos[0],pos[1]],[pos[2],pos[3]]]))
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)
row = 4
col = 3
fig = plt.figure()
fig.set_size_inches(9,9.5)
matplotlib.rcParams.update({'font.size': 7}) 


fig.text(x=0.5, y=0.915, s='Observations', fontsize=10, fontweight='bold', ha='center')
fig.text(x=0.5, y=0.45, s='Hindcast model experiment', fontsize=10, fontweight='bold', ha='center')

##########################################################################################
ax = plt.subplot2grid((row, col), (0, 0), rowspan=1, colspan=1)
pos = ax.get_position()
print(pos)
pos_plot = [pos.x0, pos.y0+0.04, pos.x1, pos.y1] 
print(pos_plot)

temp_slopes_40 = sea_surface_temperature_slopes_82 * 10
levels = arange(-0.8,0.8,0.1)
plot = plt.contourf(temp_slopes_40.lon, temp_slopes_40.lat, temp_slopes_40,
            cmap='seismic', levels=levels, extend='both')
ax.set_position(bbox(pos_plot))

# plt.text(120,-29,'SAB',color='k',fontsize=10)
# plt.text(147,-30,'TS',color='k',fontsize=10)
plt.plot(
    [lon_SA[0], lon_SA[1], lon_SA[1], lon_SA[0], lon_SA[0]],
    [lat_SA[0], lat_SA[0], lat_SA[1], lat_SA[1], lat_SA[0]],
    color='k', linestyle='--', linewidth=2)
plt.plot(
    [lon_TS[0], lon_TS[1], lon_TS[1], lon_TS[0], lon_TS[0]],
    [lat_TS[0], lat_TS[0], lat_TS[1], lat_TS[1], lat_TS[0]],
    color='k', linestyle='--', linewidth=2)

ax.set_facecolor('gray')
plt.title(r'a) $SST$ 1982 to 2017 trend', loc='left')
plt.xlabel('Longitude ($^{\circ}E$)')
plt.xticks(arange(100,160,10))
# plt.ylabel('Latitude ($^{\circ}N$)')
plt.gca().set_yticklabels('')
plt.grid(linewidth=0.5)

axins = inset_axes(ax,
                   width="100%",  # width = 5% of parent_bbox width
                   height="6.5%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(0, -0.45, 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0)
cbar = plt.colorbar(plot, orientation='horizontal',
                    label=r'$SST\ (^{\circ}C \cdot 10\ yrs^{-1})$', cax=axins)
##########################################################################################


##########################################################################################
ax = plt.subplot2grid((row, col), (0, 1), rowspan=1, colspan=1)
pos = ax.get_position()
print(pos)
pos_plot = [pos.x0, pos.y0+0.04, pos.x1, pos.y1] 
print(pos_plot)

temp_slopes_40 = sea_surface_temperature_slopes_93 * 10
levels = arange(-0.8,0.8,0.1)
plot = plt.contourf(temp_slopes_40.lon, temp_slopes_40.lat, temp_slopes_40,
            cmap='seismic', levels=levels, extend='both')
ax.set_position(bbox(pos_plot))

plt.text(120,-29,'SAB',color='k',fontsize=10)
plt.text(147,-30,'TS',color='k',fontsize=10)
plt.plot(
    [lon_SA[0], lon_SA[1], lon_SA[1], lon_SA[0], lon_SA[0]],
    [lat_SA[0], lat_SA[0], lat_SA[1], lat_SA[1], lat_SA[0]],
    color='k', linestyle='--', linewidth=2)
plt.plot(
    [lon_TS[0], lon_TS[1], lon_TS[1], lon_TS[0], lon_TS[0]],
    [lat_TS[0], lat_TS[0], lat_TS[1], lat_TS[1], lat_TS[0]],
    color='k', linestyle='--', linewidth=2)

ax.set_facecolor('gray')
plt.title(r'b) $SST$ 1993 to 2017 trend', loc='left')
plt.xlabel('Longitude ($^{\circ}E$)')
plt.xticks(arange(100,160,10))
# plt.ylabel('Latitude ($^{\circ}N$)')
plt.gca().set_yticklabels('')
plt.grid(linewidth=0.5)

axins = inset_axes(ax,
                   width="100%",  # width = 5% of parent_bbox width
                   height="6.5%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(0, -0.45, 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0)
cbar = plt.colorbar(plot, orientation='horizontal',
                    label=r'$SST\ (^{\circ}C \cdot 10\ yrs^{-1})$', cax=axins)
##########################################################################################


##########################################################################################
ax = plt.subplot2grid((row, col), (0, 2), rowspan=1, colspan=1)
pos = ax.get_position()
print(pos)
pos_plot = [pos.x0, pos.y0+0.04, pos.x1, pos.y1] 
print(pos_plot)

eta_t_slopes_40 = GSLA_slopes * 10
levels = arange(-10,10,2)
plot = plt.contourf(eta_t_slopes_40.lon, eta_t_slopes_40.lat, eta_t_slopes_40,
            cmap='BrBG', levels=levels, extend='both')
ax.set_position(bbox(pos_plot))

plt.plot(
    [lon_TS[0], lon_TS[1], lon_TS[1], lon_TS[0], lon_TS[0]],
    [lat_TS[0], lat_TS[0], lat_TS[1], lat_TS[1], lat_TS[0]],
    color='k', linestyle='--', linewidth=2)

ax.set_facecolor('gray')
plt.title(r'c) $\eta$ 1993 to 2017 trend', loc='left')
plt.xlabel('Longitude ($^{\circ}E$)')
plt.xticks(arange(100,160,20))
# plt.ylabel('Latitude ($^{\circ}N$)')
plt.gca().set_yticklabels('')
plt.grid(linewidth=0.5)

axins = inset_axes(ax,
                   width="100%",  # width = 5% of parent_bbox width
                   height="6.5%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(0, -0.45, 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0)
cbar = plt.colorbar(plot, orientation='horizontal',
                    label=r'$\eta\ (cm \cdot 10\ yrs^{-1})$', cax=axins)
##########################################################################################


##########################################################################################
ax = plt.subplot2grid((row, col), (1, 0), rowspan=1, colspan=1)
pos = ax.get_position()
print(pos)
pos_plot = [pos.x0, pos.y0, pos.x1-0.01, pos.y1-0.05] 
print(pos_plot)

plot = sea_surface_temperature_anom_SA_82.plot(color='k', linewidth=0.75)
ax.set_position(bbox(pos_plot))

plt.plot(
    [years_SST[0], years_SST[-1]], 
    [years_SST[0]*sea_surface_temperature_anom_SA_slope_82+sea_surface_temperature_anom_SA_intrsct_82, 
     years_SST[-1]*sea_surface_temperature_anom_SA_slope_82+sea_surface_temperature_anom_SA_intrsct_82], 
    color='gray', linewidth=0.5)

plt.plot(
    [arange(1993,2017)[0], arange(1993,2017)[-1]], 
    [arange(1993,2017)[0]*sea_surface_temperature_anom_SA_slope_93+sea_surface_temperature_anom_SA_intrsct_93, 
     arange(1993,2017)[-1]*sea_surface_temperature_anom_SA_slope_93+sea_surface_temperature_anom_SA_intrsct_93], 
    color='gray', linewidth=0.5)

# ax.set_facecolor('gray')
plt.title('')
plt.title('d) $SST$ in SAB', loc='left')
plt.xlabel('Time (year)')
plt.xticks(arange(1983,2017,10), rotation=0)
plt.ylabel(r'$SST$ ($^{\circ}C$)', labelpad=-5)
plt.yticks(arange(-0.5,0.5,0.25))
plt.grid(linewidth=0.5)
##########################################################################################


##########################################################################################
ax = plt.subplot2grid((row, col), (1, 1), rowspan=1, colspan=1)
pos = ax.get_position()
print(pos)
pos_plot = [pos.x0, pos.y0, pos.x1-0.01, pos.y1-0.05] 
print(pos_plot)

plot = sea_surface_temperature_anom_TS_82.plot(color='k', linewidth=0.75)
ax.set_position(bbox(pos_plot))

plt.plot(
    [years_SST[0], years_SST[-1]], 
    [years_SST[0]*sea_surface_temperature_anom_TS_slope_82+sea_surface_temperature_anom_TS_intrsct_82, 
     years_SST[-1]*sea_surface_temperature_anom_TS_slope_82+sea_surface_temperature_anom_TS_intrsct_82], 
    color='gray', linewidth=0.5)

plt.plot(
    [arange(1993,2017)[0], arange(1993,2017)[-1]], 
    [arange(1993,2017)[0]*sea_surface_temperature_anom_TS_slope_93+sea_surface_temperature_anom_TS_intrsct_93, 
     arange(1993,2017)[-1]*sea_surface_temperature_anom_TS_slope_93+sea_surface_temperature_anom_TS_intrsct_93], 
    color='gray', linewidth=0.5)

# ax.set_facecolor('gray')
plt.title('')
plt.title('e) $SST$ in TS', loc='left')
plt.xlabel('Time (year)')
plt.xticks(arange(1983,2017,10), rotation=0)
plt.ylabel(r'$SST$ ($^{\circ}C$)', labelpad=-5)
plt.yticks(arange(-1,1,0.5))
plt.grid(linewidth=0.5)
##########################################################################################


##########################################################################################
ax = plt.subplot2grid((row, col), (1, 2), rowspan=1, colspan=1)
pos = ax.get_position()
print(pos)
pos_plot = [pos.x0, pos.y0, pos.x1-0.01, pos.y1-0.05] 
print(pos_plot)

plot = GSLA_anom_TS.plot(color='k', linewidth=0.75)
ax.set_position(bbox(pos_plot))

plt.plot(
    [years[0], years[-1]], 
    [years[0]*GSLA_anom_TS_slope+GSLA_anom_TS_intrsct, years[-1]*GSLA_anom_TS_slope+GSLA_anom_TS_intrsct], 
    color='gray', linewidth=0.5)

# ax.set_facecolor('gray')
plt.title(r'f) $\eta$ in TS', loc='left')
plt.xlabel('Time (year)')
plt.xticks(arange(1983,2017,10), rotation=0)
plt.ylabel(r'$\eta$ (cm)', labelpad=-5)
plt.yticks(arange(-6,6,3))
plt.grid(linewidth=0.5)
##########################################################################################



##########################################################################################
ax = plt.subplot2grid((row, col), (2, 0), rowspan=1, colspan=1)
pos = ax.get_position()
print(pos)
pos_plot = [pos.x0, pos.y0-0.03, pos.x1, pos.y1-0.07] 
print(pos_plot)

temp_slopes_40 = temp_slopes_82 * 10
levels = arange(-0.8,0.8,0.1)
plot = plt.contourf(temp_slopes_40.longitude, temp_slopes_40.latitude, temp_slopes_40,
            cmap='seismic', levels=levels, extend='both')
ax.set_position(bbox(pos_plot))

# plt.text(120,-29,'SAB',color='k',fontsize=10)
# plt.text(147,-30,'TS',color='k',fontsize=10)
plt.plot(
    [xt_ocean_SA[0], xt_ocean_SA[1], xt_ocean_SA[1], xt_ocean_SA[0], xt_ocean_SA[0]],
    [yt_ocean_SA[0], yt_ocean_SA[0], yt_ocean_SA[1], yt_ocean_SA[1], yt_ocean_SA[0]],
    color='k', linestyle='--', linewidth=2)
plt.plot(
    [xt_ocean_TS[0], xt_ocean_TS[1], xt_ocean_TS[1], xt_ocean_TS[0], xt_ocean_TS[0]],
    [yt_ocean_TS[0], yt_ocean_TS[0], yt_ocean_TS[1], yt_ocean_TS[1], yt_ocean_TS[0]],
    color='k', linestyle='--', linewidth=2)

ax.set_facecolor('gray')
plt.title(r'g) $SST$ 1982 to 2017 trend', loc='left')
plt.xlabel('')
plt.xlabel('Longitude ($^{\circ}E$)')
plt.xticks(arange(100,160,10))
# plt.ylabel('yt_oceanitude ($^{\circ}N$)')
plt.gca().set_yticklabels('')
plt.grid(linewidth=0.5)

axins = inset_axes(ax,
                   width="100%",  # width = 5% of parent_bbox width
                   height="6.5%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(0, -0.45, 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0)
cbar = plt.colorbar(plot, orientation='horizontal',
                    label=r'$SST\ (^{\circ}C \cdot 10\ yrs^{-1})$', cax=axins)
##########################################################################################


##########################################################################################
ax = plt.subplot2grid((row, col), (2, 1), rowspan=1, colspan=1)
pos = ax.get_position()
print(pos)
pos_plot = [pos.x0, pos.y0-0.03, pos.x1, pos.y1-0.07] 
print(pos_plot)

temp_slopes_40 = temp_slopes_93 * 10
levels = arange(-0.8,0.8,0.1)
plot = plt.contourf(temp_slopes_40.longitude, temp_slopes_40.latitude, temp_slopes_40,
            cmap='seismic', levels=levels, extend='both')
ax.set_position(bbox(pos_plot))

# plt.text(120,-29,'SAB',color='k',fontsize=10)
# plt.text(147,-30,'TS',color='k',fontsize=10)
plt.plot(
    [xt_ocean_SA[0], xt_ocean_SA[1], xt_ocean_SA[1], xt_ocean_SA[0], xt_ocean_SA[0]],
    [yt_ocean_SA[0], yt_ocean_SA[0], yt_ocean_SA[1], yt_ocean_SA[1], yt_ocean_SA[0]],
    color='k', linestyle='--', linewidth=2)
plt.plot(
    [xt_ocean_TS[0], xt_ocean_TS[1], xt_ocean_TS[1], xt_ocean_TS[0], xt_ocean_TS[0]],
    [yt_ocean_TS[0], yt_ocean_TS[0], yt_ocean_TS[1], yt_ocean_TS[1], yt_ocean_TS[0]],
    color='k', linestyle='--', linewidth=2)

ax.set_facecolor('gray')
plt.title(r'h) $SST$ 1993 to 2017 trend', loc='left')
plt.xlabel('')
plt.xlabel('Longitude ($^{\circ}E$)')
plt.xticks(arange(100,160,10))
# plt.ylabel('yt_oceanitude ($^{\circ}N$)')
plt.gca().set_yticklabels('')
plt.grid(linewidth=0.5)

axins = inset_axes(ax,
                   width="100%",  # width = 5% of parent_bbox width
                   height="6.5%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(0, -0.45, 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0)
cbar = plt.colorbar(plot, orientation='horizontal',
                    label=r'$SST\ (^{\circ}C \cdot 10\ yrs^{-1})$', cax=axins)
##########################################################################################


##########################################################################################
ax = plt.subplot2grid((row, col), (2, 2), rowspan=1, colspan=1)
pos = ax.get_position()
print(pos)
pos_plot = [pos.x0, pos.y0-0.03, pos.x1, pos.y1-0.07] 
print(pos_plot)

eta_t_slopes_40 = eta_t_slopes * 10
levels = arange(-10,10,2)
plot = plt.contourf(eta_t_slopes_40.longitude, eta_t_slopes_40.latitude, eta_t_slopes_40,
            cmap='BrBG', levels=levels, extend='both')
ax.set_position(bbox(pos_plot))

plt.plot(
    [xt_ocean_TS[0], xt_ocean_TS[1], xt_ocean_TS[1], xt_ocean_TS[0], xt_ocean_TS[0]],
    [yt_ocean_TS[0], yt_ocean_TS[0], yt_ocean_TS[1], yt_ocean_TS[1], yt_ocean_TS[0]],
    color='k', linestyle='--', linewidth=2)

ax.set_facecolor('gray')
plt.title(r'i) $\eta$ 1993 to 2017 trend', loc='left')
plt.xlabel('')
plt.xlabel('Longitude ($^{\circ}E$)')
plt.xticks(arange(100,160,20))
# plt.ylabel('yt_oceanitude ($^{\circ}N$)')
plt.gca().set_yticklabels('')
plt.grid(linewidth=0.5)

axins = inset_axes(ax,
                   width="100%",  # width = 5% of parent_bbox width
                   height="6.5%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(0, -0.45, 1, 1),
                   bbox_transform=ax.transAxes,
                   borderpad=0)
cbar = plt.colorbar(plot, orientation='horizontal',
                    label=r'$\eta\ (cm \cdot 10\ yrs^{-1})$', cax=axins)
##########################################################################################


##########################################################################################
ax = plt.subplot2grid((row, col), (3, 0), rowspan=1, colspan=1)
pos = ax.get_position()
print(pos)
pos_plot = [pos.x0, pos.y0-0.07, pos.x1-0.01, pos.y1-0.12]
print(pos_plot)

plot = temp_anom_SA_82.plot(color='k', linewidth=0.75)
ax.set_position(bbox(pos_plot))

plt.plot(
    [years_SST[0], years_SST[-1]], 
    [years_SST[0]*temp_anom_SA_slope_82+temp_anom_SA_intrsct_82, 
     years_SST[-1]*temp_anom_SA_slope_82+temp_anom_SA_intrsct_82], 
    color='gray', linewidth=0.5)

plt.plot(
    [arange(1993,2017)[0], arange(1993,2017)[-1]], 
    [arange(1993,2017)[0]*temp_anom_SA_slope_93+temp_anom_SA_intrsct_93, 
     arange(1993,2017)[-1]*temp_anom_SA_slope_93+temp_anom_SA_intrsct_93], 
    color='gray', linewidth=0.5)

# ax.set_facecolor('gray')
plt.title('')
plt.title('j) $SST$ in SAB', loc='left')
plt.xlabel('Time (year)')
plt.xticks(arange(1983,2017,10), rotation=0)
plt.ylabel(r'$SST$ ($^{\circ}C$)', labelpad=-5)
plt.yticks(arange(-0.5,0.5,0.25))
plt.grid(linewidth=0.5)
##########################################################################################


##########################################################################################
ax = plt.subplot2grid((row, col), (3, 1), rowspan=1, colspan=1)
pos = ax.get_position()
print(pos)
pos_plot = [pos.x0, pos.y0-0.07, pos.x1-0.01, pos.y1-0.12]
print(pos_plot)

plot = temp_anom_TS_82.plot(color='k', linewidth=0.75)
ax.set_position(bbox(pos_plot))

plt.plot(
    [years_SST[0], years_SST[-1]], 
    [years_SST[0]*temp_anom_TS_slope_82+temp_anom_TS_intrsct_82, 
     years_SST[-1]*temp_anom_TS_slope_82+temp_anom_TS_intrsct_82], 
    color='gray', linewidth=0.5)

plt.plot(
    [arange(1993,2017)[0], arange(1993,2017)[-1]], 
    [arange(1993,2017)[0]*temp_anom_TS_slope_93+temp_anom_TS_intrsct_93, 
     arange(1993,2017)[-1]*temp_anom_TS_slope_93+temp_anom_TS_intrsct_93], 
    color='gray', linewidth=0.5)

# ax.set_facecolor('gray')
plt.title('')
plt.title('k) $SST$ in TS', loc='left')
plt.xlabel('Time (year)')
plt.xticks(arange(1983,2017,10), rotation=0)
plt.ylabel(r'$SST$ ($^{\circ}C$)', labelpad=-5)
plt.yticks(arange(-1,1,0.5))
plt.grid(linewidth=0.5)
##########################################################################################


##########################################################################################
ax = plt.subplot2grid((row, col), (3, 2), rowspan=1, colspan=1)
pos = ax.get_position()
print(pos)
pos_plot = [pos.x0, pos.y0-0.07, pos.x1-0.01, pos.y1-0.12]
print(pos_plot)

plot = eta_t_anom_TS.plot(color='k', linewidth=0.75)
ax.set_position(bbox(pos_plot))

plt.plot(
    [years[0], years[-1]], 
    [years[0]*eta_t_anom_TS_slope+eta_t_anom_TS_intrsct, years[-1]*eta_t_anom_TS_slope+eta_t_anom_TS_intrsct], 
    color='gray', linewidth=0.5)

# ax.set_facecolor('gray')
plt.title(r'l) $\eta$ in TS', loc='left')
plt.xlabel('Time (year)')
plt.xticks(arange(1983,2017,10), rotation=0)
plt.ylabel(r'$\eta$ (cm)', labelpad=-5)
plt.yticks(arange(-6,6,3))
plt.grid(linewidth=0.5)
##########################################################################################



output_ls = os.listdir(figures_path)
if not script_name:
    script_name = 'test'
elif script_name not in output_ls:
    os.mkdir(figures_path + '/' + script_name)
plt.savefig(figures_path + '/' + script_name + '/' + script_name[0:3] \
            + '_fig1_.png', bbox_inches='tight', dpi=300)